<a href="https://colab.research.google.com/github/GTuritto/Demos4Course/blob/main/Demo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projecto Documentador de Software

**Instalamos los paquetes requeridos**


In [ ]:
!pip install pyautogen dask[dataframe] mistralai

**Importamos las dependencias necesarias**

In [1]:
from IPython.display import Image, display
from google.colab import userdata

import autogen
import os
import json
from google.colab import userdata

Configuramos la conexion al API del LLM

In [2]:
# API Configuracion para OpenAI
# apiConfig = [{
#        "model": "gpt-4o-mini",
#        "max_tokens": 500,
#        "api_key": userdata.get('OPENAI_API_KEY')
#    }]


# API Configuracion para Mistral -- Preferimos usar esta para las pruebas ya que no hay que pagar y es tan buena como la de OpenAI
api_config = [{
        "model": "mistral-large-latest",
        "max_tokens": 2000,
        "api_key": userdata.get('MISTRAL_API_KEY'),
        "api_type": "mistral"
    }]

# Configuración para el LLM, incluyendo lógica de reintentos y semilla de caché para consistencia
llm_config = {
    "config_list": api_config,
    "cache_seed": 42,  # Semilla de caché para un comportamiento consistente
    "retry_on_rate_limit": True,  # Reintentar si se alcanza el límite de la API
    "retry_on_timeout": True,     # Reintentar si hay un tiempo de espera
    "max_retries": 5,             # Número máximo de reintentos
    "retry_delay": 0.3,           # Retraso más corto para reintentos más rápidos
}

## Creemos los Agentes

Aqui controlaremos la creatividad de los agentes, ya que queremos que el Product Manager se un poco mas creativo que el resto

### User Proxy

El agente que servira del proxy para la iteracion con el usuario

In [3]:
# User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE"
)

### Niveles de Creatividad (temperature)

In [4]:
# Define custom temperature for each agent

llm_config_high = llm_config.copy()
llm_config_high["temperature"] = 0.9 # More creative than usual

llm_config_low = llm_config.copy()
llm_config_low["temperature"] = 0.5 # Less creative, more structured

llm_config_normal = llm_config.copy()
llm_config_normal["temperature"] = 0.7 # Normal level of creativity

### Agente Product Manager

Agente que actua como un Product Manager

In [5]:
# Product Manager Agent
pm = autogen.AssistantAgent(
    name="Product_Manager",
    system_message="You are an amazing Product Manager, creative in software product ideas, with knowledge in multiples verticals",
    llm_config=llm_config_high,
)


### Agente Business Analyst

Agente que actua como un Business Analyst

In [6]:
# Business Analyst Agent
ba = autogen.AssistantAgent(
    name="Business_Analyst",
    system_message="Act as an Expert in business analysis and requirements gathering.",
    llm_config=llm_config_low,
)

### Agente Architect

Agente que actua como un Arquitecto de Software

In [7]:
# Architect Agent
architect = autogen.AssistantAgent(
    name="Architect",
    system_message="You are a Software Architect, with expereience in creating escalable and solid architectures, you have knowledge of Microservices and distributed monolith, as well architecture for FrontEnd in Web and Mobile. You are responsible for defining the architecture and design decisions.",
    llm_config=llm_config_normal,
)

### Agente Coder

Agente que actua como un Ingeniero de Software

In [8]:
# Coder Assistant Agent
coder = autogen.AssistantAgent(
    name="Coder",
    system_message="Act as a Staff Software Engineer.",
    llm_config=llm_config_normal,
)

### Agente Analista de QA

Agente que actua como un experto en **QA**

In [10]:
# QA Agent
qa = autogen.AssistantAgent(
    name="QA",
    system_message="Expert in testing strategies, risk analysis and creating BDD specifications. You are responsible to create the Testing Plan, y Gerhkin Specifications",
    llm_config=llm_config_low,
)

### Agente generador de Documentacion

Este agente esta encargdo de generar el Documento final, combinando los documentos de cada uno de los integrantes, coordinacion de la generacion de documentos y revisa y genera recomendacion para mejorar el documento

In [11]:
# Documentation Agent

class DocumentationAgent(autogen.AssistantAgent):
    def __init__(self, name, llm_config):
        super().__init__(name=name, llm_config=llm_config)
        self.final_document = {}

    def contribute(self, agent_name, section_title, content):
        # Each agent contributes to the final document
        self.final_document[agent_name] = {
            "section_title": section_title,
            "content": content
        }

    def generate_final_document(self):
        # Create documentation folder if it doesn't exist
        if not os.path.exists("documentation"):
            os.makedirs("documentation")

        # Combine all contributions into a final document
        final_content = "# Final Documentation\n\n"
        for agent_name, section in self.final_document.items():
            final_content += f"## {section['section_title']}\n\n{section['content']}\n\n"

        # Write the Markdown file
        markdown_path = "documentation/final_document.md"
        with open(markdown_path, "w") as md_file:
            md_file.write(final_content)

        print(f"Final document created in 'documentation' folder:\n- {markdown_path}")
        return markdown_path

    def self_reflect_and_improve(self, agent_name):
        # Each agent self-reflects on their contribution and suggests improvements
        reflection = f"{agent_name} self-reflects and suggests improvements on their section."
        print(f"{agent_name}: {reflection}\n")
        self.final_document[agent_name]["content"] += f"\n\n**Self-Reflection and Improvements:**\n\n{reflection}"

    def peer_review(self, agent_name, reviewer_name):
        # Each agent reviews the contribution of another agent and suggests improvements
        review = f"{reviewer_name} reviews {agent_name}'s section and suggests improvements."
        print(f"{reviewer_name}: {review}\n")
        self.final_document[agent_name]["content"] += f"\n\n**Peer Review by {reviewer_name}:**\n\n{review}"

# Instantiate the Documentation Agent
doc_agent = DocumentationAgent(
    name="Documentation_Agent",
    llm_config=llm_config_normal
)


## Creacion del Chat de grupo de Agentes

In [12]:
# GroupChat Setup with all agents
groupchat = autogen.GroupChat(agents=[user_proxy, pm, ba, architect, coder, qa, doc_agent], messages=[], max_round=12)

# GroupChat Manager to orchestrate the chat
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


## Fases de Colaboración

En esta seccion definiremos la coordinacion entre Agentes.

Hemos dividido el proceso en tres fases de colaboración, la primera fase es una colaboración entre los agentes Product Manager y el Business Analyst, ellos generara un Documento que sera usado en la siguiente fase.
La segunda fase es una colaboración entre los agentes Coder, Architect y QA, en esta fase colaboraran y ampliaran añadiendo su conocimiento sobre el documento creado por los agenes Product Manager y Business Analyst.
La ultima y tercera fase, es es Agente de Documentación recopilar todo en un unico documento revisar y hacer sugerencias a cada uno de los agentes anteriores para revisar el contenido.

Todo termina cuando no hay mas sugerencias que hacer.

### Fase 1

In [13]:
# Function to simulate the collaboration in Phase 1
def phase_1_collaboration():
    print("Phase 1: Product Manager and Business Analyst Collaboration\n")

    # Product Manager and Business Analyst collaborate
    for agent in [pm, ba]:
        response = agent.generate_reply(groupchat.messages)
        content = response.get('content', '') if isinstance(response, dict) else response

        if agent.name == "Product_Manager":
            doc_agent.contribute(agent.name, "Product Requirements Document (PRD)", content)
        elif agent.name == "Business_Analyst":
            doc_agent.contribute(agent.name, "Business Requirements and Use Cases", content)

        print(f"{agent.name} contributed to the document in Phase 1.\n")


### Fase 2

In [14]:
# Function to simulate the collaboration in Phase 2
def phase_2_collaboration():
    print("Phase 2: Coder, Architect, and QA Collaboration\n")

    # Coder and Architect work together, and QA works in parallel
    for agent in [coder, architect, qa]:
        response = agent.generate_reply(groupchat.messages)
        content = response.get('content', '') if isinstance(response, dict) else response

        if agent.name == "Coder":
            doc_agent.contribute(agent.name, "Implementation Plan", content)
        elif agent.name == "Architect":
            doc_agent.contribute(agent.name, "Architectural Decisions", content)
        elif agent.name == "QA":
            doc_agent.contribute(agent.name, "Testing Plan and BDD Specifications", content)

        print(f"{agent.name} contributed to the document in Phase 2.\n")


### Fase3

In [15]:
# Function to simulate the final revision in Phase 3
def phase_3_revision():
    print("Phase 3: Final Revision and Improvement\n")

    # Self-reflection and peer review
    for agent in groupchat.agents:
        if agent.name != "User_proxy" and agent.name != "Documentation_Agent":
            doc_agent.self_reflect_and_improve(agent.name)

    for agent in groupchat.agents:
        if agent.name != "User_proxy" and agent.name != "Documentation_Agent":
            for reviewer in groupchat.agents:
                if reviewer.name != "User_proxy" and reviewer.name != agent.name and reviewer.name != "Documentation_Agent":
                    doc_agent.peer_review(agent.name, reviewer.name)


## Activacion

Aqui definimos que deben de documentar todos estos agentes y empezamos cada una de las fases

In [ ]:
# Function to start the conversation and generate the final document
def start_conversation():
    initial_message = "We are designing a new clone of Jira, we want to include AI in order to help with the creation of Tickets and assigning to the right person."

    groupchat.messages.append({
        "sender": user_proxy.name,
        "content": initial_message,
        "role": "user"
    })

    # Phase 1: Product Manager and Business Analyst Collaboration
    phase_1_collaboration()

    # Phase 2: Coder, Architect, and QA Collaboration
    phase_2_collaboration()

    # Phase 3: Final Revision and Improvement
    phase_3_revision()

    # Generate and deliver the final documentation
    doc_agent.generate_final_document()

# Start the group conversation
start_conversation()




---

